In [98]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.transforms import v2
import random
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import ConcatDataset

import tqdm
import torch.nn as nn
import torch.optim as optim
import torchvision
import os
from sklearn.metrics import accuracy_score


import torch
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from glob import glob
from keras import optimizers
from keras.layers import GlobalMaxPooling2D
from keras.models import load_model



In [99]:
random.seed(1)

# Define transforms
transform = transforms.Compose([
    transforms.ToTensor()
])

# Load images from the folder
folder_train = ImageFolder('./train', transform=transform)
folder_val = ImageFolder('./val', transform=transform)

# Create a DataLoader
train_loader = DataLoader(folder_train, batch_size=8, shuffle=True)
val_loader = DataLoader(folder_val, batch_size=8, shuffle=True)

folder_test = ImageFolder('./test', transform=transform)
test_loader = DataLoader(folder_test, batch_size=8, shuffle=True)

# Get a batch of examples
Xexamples, Yexamples = next(iter(train_loader))

# Plot the examples
plt.figure(figsize=(10, 5))
for i in range(8):
    plt.subplot(2, 4, i+1)
    img = Xexamples[i].numpy().transpose(1, 2, 0)
    plt.imshow(img, interpolation='none')
    plt.title('NV' if Yexamples[i] == 0 else 'MEL')  # Assuming Yexamples is binary (0 or 1)
    plt.xticks([])
    plt.yticks([])

plt.show()



# train sample

In [100]:
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler


# Shuffle the indices of the dataset
indices = np.arange(len(folder_train))
np.random.shuffle(indices)

# Select the first 100 indices
subset_indices = indices[:100]

# Create a DataLoader for the subset
train_loader_subset = DataLoader(folder_train, batch_size=8, sampler=SubsetRandomSampler(subset_indices))


# Get a batch of examples
Xexamples, Yexamples = next(iter(train_loader_subset))

# Plot the examples
plt.figure(figsize=(10, 5))
for i in range(8):
    plt.subplot(2, 4, i+1)
    img = Xexamples[i].numpy().transpose(1, 2, 0)
    plt.imshow(img, interpolation='none')
    plt.title('NV' if Yexamples[i] == 0 else 'MEL')  # Assuming Yexamples is binary (0 or 1)
    plt.xticks([])
    plt.yticks([])

plt.show()



# Training and Validation

In [101]:
def evaluate_during_training(model, val_loader):
    model.eval()  # Set the model in evaluation mode
    all_gold = []
    all_pred = []

    with torch.no_grad():
        for Xbatch, Ybatch in val_loader:
            outputs = model(Xbatch)
            predictions = torch.round(outputs).squeeze()  # Assuming binary classification
            all_gold.extend(Ybatch.numpy())
            all_pred.extend(predictions.numpy())

    accuracy = accuracy_score(all_gold, all_pred)
    return accuracy


In [102]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs, save_path):
    best_val_acc = 0

    for epoch in range(num_epochs):
        running_loss = 0.0
        model.eval()
        for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False):
            optimizer.zero_grad()
            outputs = model(images)

            # Calculate the loss
            loss = criterion(outputs.squeeze(), labels.float())

            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)  # Calculate average loss per sample
        val_acc = evaluate_during_training(model, val_loader)  # Evaluate on validation set

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc

            torch.save(model.state_dict(), save_path)
            print(f"Model saved at {save_path}")

# Baseline

In [16]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import torch as torch


class Baseline_CNN(nn.Module):
    def __init__(self):
        super(Baseline_CNN, self).__init__()

        self.features = nn.Sequential(
            # 3: image input channel,3 channels RGB, Red Green Blue
            # 32 filters in the convoluational layer, output feature map = 32
            # relu for nonzero numbers to introduce non-linearity, helps mitigating the vanishing gradient problem
            # pooling to reduce spatial dimension

            nn.Conv2d(3, 32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=8),
            nn.Conv2d(64, 128, kernel_size=3),
            nn.ReLU(),
            nn.Flatten()
        )

        # Calculate the output size after the convolutional layers
        self.classifier = nn.Sequential(
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()

        )

    def forward(self, x):
      # print("Input size:", x.size())
      x = self.features(x)
      # print("After features size:", x.size())
      x = self.classifier(x)
      # print("Output size:", x.size())
      return x


In [ ]:
# Constants
learning_rate = 0.01
num_epochs = 10
save_path = "trained_models/baseline.pth"

# Initialize model, loss function, and optimizer
model_baseline = Baseline_CNN()
criterion = nn.BCELoss() # Binary Cross Entropy Loss
optimizer = optim.Adam(model_baseline.parameters(), lr=learning_rate)

# Call the training function
train_model(model_baseline, criterion, optimizer, train_loader, val_loader, num_epochs, save_path)

In [17]:
save_path = "trained_models/baseline.pth"
model_baseline = Baseline_CNN()
model_baseline.load_state_dict(torch.load(save_path))
model_baseline.eval()

val_acc = evaluate_during_training(model_baseline, val_loader)  # Evaluate on validation set
print("Baseline_CNN validation accuracy: ", val_acc)

Baseline_CNN validation accuracy:  0.7755591054313099


# Augmentation

In [20]:

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

folder_train = ImageFolder('./train', transform=transform)


R_channel = []
G_channel = []
B_channel = []


for img, _ in folder_train:
    img = img.numpy()
    R_channel.append(np.mean(img[0, :, :]))
    G_channel.append(np.mean(img[1, :, :]))
    B_channel.append(np.mean(img[2, :, :]))


mean = [np.mean(R_channel), np.mean(G_channel), np.mean(B_channel)]
std = [np.std(R_channel), np.std(G_channel), np.std(B_channel)]

print("Mean:", mean)
print("Std:", std)

Mean: [0.68368095, 0.5231375, 0.50991017]
Std: [0.15012987, 0.1178059, 0.12110886]


In [96]:
transform_augmented = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.RandomResizedCrop(size=(128, 128), antialias=True),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomRotation(degrees=35),  # Random rotation up to 15 degrees
    v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random color jitter
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.68368095, 0.5231375, 0.50991017], std=[0.15012987, 0.1178059, 0.12110886]),
])


folder_train = ImageFolder('./train', transform=transform)
folder_train_augmented = ImageFolder('./train', transform=transform_augmented)

combined_dataset = ConcatDataset([folder_train, folder_train_augmented])

train_loader_augmented = DataLoader(combined_dataset, batch_size=16, shuffle=True)

Xexamples, Yexamples = next(iter(train_loader_augmented))


plt.figure(figsize=(10, 5))
for i in range(8):
    plt.subplot(2, 4, i+1)
    img = Xexamples[i].numpy().transpose(1, 2, 0)
    img = np.clip(img, 0, 1)
    plt.imshow(img, interpolation='none')
    plt.title('NV' if Yexamples[i] == 0 else 'MEL')
    plt.xticks([])
    plt.yticks([])

plt.show()


In [26]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import torch as torch

class Augmented_CNN(nn.Module):
    def __init__(self):
        super(Augmented_CNN, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=8),
            nn.Conv2d(64, 128, kernel_size=3),
            nn.ReLU(),
            nn.Flatten()
        )

        self.classifier = nn.Sequential(
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout layer with 50% probability
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [103]:
learning_rate = 0.001
num_epochs = 10
save_path = "trained_models/augmented.pth"

model_augmentation = Augmented_CNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model_augmentation.parameters(), lr=learning_rate, weight_decay=0.001)

train_model(model_augmentation, criterion, optimizer, train_loader_augmented, val_loader, num_epochs, save_path)

TypeError: 'module' object is not callable

In [31]:
save_path = "trained_models/augmented_.pth"
model_augmented = Augmented_CNN()
model_augmented.load_state_dict(torch.load(save_path))
criterion = nn.BCELoss()

val_acc = evaluate_during_training(model_augmented, val_loader)
print("Augmentated_CNN validation accuracy: ", val_acc)

Augmentated_CNN validation accuracy:  0.8162939297124601


# Residual

In [34]:
class ResidualBlock(nn.Module):
    def __init__(self, num_channels):
        super(ResidualBlock, self).__init__()

        self.conv1 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        out = self.relu(out)
        return out


class Residual_CNN(nn.Module):
    def __init__(self):
        super(Residual_CNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.residual_block1 = ResidualBlock(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.residual_block2 = ResidualBlock(128)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.residual_block3 = ResidualBlock(64)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.residual_block1(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.residual_block2(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.residual_block3(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)

        x = self.fc(x)
        return torch.sigmoid(x)


In [11]:
learning_rate = 0.01
num_epochs = 10
save_path = "trained_models/residual.pth"

model_residual = Residual_CNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model_residual.parameters(), lr=learning_rate)

train_model(model_residual, criterion, optimizer, train_loader, val_loader, num_epochs, save_path)

Epoch 1/10, Loss: 49.9542, Validation Accuracy: 0.5000
Model saved at trained_models/residual.pth


Epoch 2/10, Loss: 50.0000, Validation Accuracy: 0.5000


KeyboardInterrupt: 

In [52]:
save_path = "trained_models/residual.pth"
model_residual = Residual_CNN()
model_residual.load_state_dict(torch.load(save_path))
model_baseline.eval()

val_acc = evaluate_during_training(model_residual, val_loader)
print("Residual validation accuracy: ", val_acc)

Residual validation accuracy:  0.7172523961661342


# Batch Normalization

In [35]:
class BatchNormalizedCNN(nn.Module):
  def __init__(self):
        super(BatchNormalizedCNN, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=8),
            nn.Conv2d(64, 128, kernel_size=3),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Flatten()
        )

        self.classifier = nn.Sequential(
            nn.Linear(128, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

  def forward(self, x):
      x = self.features(x)
      x = self.classifier(x)
      return x

In [77]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import torch as torch

learning_rate = 0.001
num_epochs = 10
save_path = "trained_models/batch_norm.pth"

model_batch_norm = BatchNormalizedCNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model_batch_norm.parameters(), lr=learning_rate)

train_model(model_batch_norm, criterion, optimizer, train_loader, val_loader, num_epochs, save_path)

Epoch 1/10, Loss: 0.5025, Validation Accuracy: 0.7604
Model saved at trained_models/batch_norm.pth


Epoch 2/10, Loss: 0.4280, Validation Accuracy: 0.7963
Model saved at trained_models/batch_norm.pth


Epoch 3/10, Loss: 0.3919, Validation Accuracy: 0.7979
Model saved at trained_models/batch_norm.pth


Epoch 4/10, Loss: 0.3778, Validation Accuracy: 0.8051
Model saved at trained_models/batch_norm.pth


Epoch 5/10, Loss: 0.3711, Validation Accuracy: 0.8115
Model saved at trained_models/batch_norm.pth


Epoch 6/10, Loss: 0.3641, Validation Accuracy: 0.8027


Epoch 7/10, Loss: 0.3517, Validation Accuracy: 0.7939


Epoch 8/10, Loss: 0.3435, Validation Accuracy: 0.8227
Model saved at trained_models/batch_norm.pth


Epoch 9/10, Loss: 0.3341, Validation Accuracy: 0.8371
Model saved at trained_models/batch_norm.pth


Epoch 10/10, Loss: 0.3259, Validation Accuracy: 0.8227


In [36]:
save_path = "trained_models/batch_norm.pth"
model_batch_norm = BatchNormalizedCNN()
model_batch_norm.load_state_dict(torch.load(save_path))
model_batch_norm.eval()

val_acc = evaluate_during_training(model_batch_norm, val_loader)
print("BatchNormalizedCNN validation accuracy: ", val_acc)

BatchNormalizedCNN validation accuracy:  0.8370607028753994


# Transfer Learning

In [84]:
weights_id = torchvision.models.VGG16_Weights.IMAGENET1K_V1
vggmodel = torchvision.models.vgg16(weights=weights_id)
vggmodel.eval()
vggtransforms = weights_id.transforms()

test_path = 'test'

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # VGG16 expects input size of 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Imagenet normalization
])

folder_test = ImageFolder('./test', transform=transform)
test_loader = DataLoader(folder_test, batch_size=8, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vggmodel.to(device)

def evaluate(model, test_loader):
    correct = 0
    total = 0
    predictions = []
    ground_truths = []
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating"):
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            ground_truths.extend(labels.cpu().numpy())
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy, predictions, ground_truths

# Evaluate the model
accuracy, predictions, ground_truths = evaluate(vggmodel, test_loader)
print("Accuracy:", accuracy)

# Print predictions
print("Predictions:", predictions)
print("Ground Truths:", ground_truths)

Evaluating: 100%|██████████| 171/171 [02:00<00:00,  1.42it/s]

Accuracy: 0.0
Predictions: [961, 551, 329, 551, 78, 78, 333, 78, 29, 961, 633, 78, 967, 963, 29, 549, 961, 78, 885, 78, 53, 78, 78, 78, 78, 961, 952, 333, 78, 961, 78, 629, 78, 78, 911, 419, 78, 78, 310, 70, 78, 78, 967, 78, 78, 963, 78, 78, 78, 29, 961, 551, 310, 29, 957, 5, 78, 78, 952, 714, 78, 419, 78, 78, 78, 78, 78, 549, 78, 969, 961, 78, 78, 629, 78, 633, 78, 78, 957, 78, 78, 78, 957, 70, 551, 969, 957, 78, 107, 714, 852, 78, 78, 341, 952, 116, 419, 78, 78, 961, 333, 78, 29, 78, 419, 334, 969, 78, 78, 78, 967, 969, 714, 78, 78, 78, 111, 957, 714, 957, 78, 78, 961, 629, 952, 111, 714, 78, 629, 78, 78, 78, 78, 111, 332, 712, 78, 78, 644, 957, 818, 78, 78, 952, 551, 78, 78, 78, 551, 78, 332, 629, 78, 567, 78, 81, 78, 957, 126, 78, 957, 666, 78, 78, 551, 910, 78, 78, 70, 974, 78, 66, 551, 78, 551, 78, 78, 78, 78, 885, 78, 961, 111, 332, 714, 969, 333, 78, 314, 78, 78, 78, 78, 78, 78, 419, 551, 78, 78, 957, 78, 78, 969, 952, 78, 78, 78, 769, 78, 78, 78, 333, 78, 78, 852, 333, 78, 332

In [64]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from keras import optimizers
from keras.layers import GlobalMaxPooling2D

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


IMAGE_SIZE = [128, 128]

weights_id = torchvision.models.VGG16_Weights.IMAGENET1K_V1
vggmodel = torchvision.models.vgg16(weights=weights_id)


vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights=weights_id, include_top=False)
for layer in vgg.layers:
  layer.trainable = False

x = Flatten()(vgg.output)
prediction = Dense(1, activation='sigmoid')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

adam = optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])


Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0   

In [65]:
train_path = 'train'
val_path = 'val'
batch_size = 8

train_datagen_augmented = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


train_datagen_normal = ImageDataGenerator(
    preprocessing_function=preprocess_input)



val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)


train_set_augmented = train_datagen_augmented.flow_from_directory(
                                              train_path,
                                              target_size=(128, 128),
                                              batch_size=batch_size,
                                              class_mode='binary',
                                              shuffle=True)

train_set_normal = train_datagen_normal.flow_from_directory(train_path,
                                              target_size=(128, 128),
                                              batch_size=batch_size,
                                              class_mode='binary',
                                              shuffle=True)


# Concatenate augmented and normal data
train_data_augmented, train_labels_augmented = next(train_set_augmented)
train_data_normal, train_labels_normal = next(train_set_normal)

combined_train_data = np.concatenate((train_data_augmented, train_data_normal))
combined_train_labels = np.concatenate((train_labels_augmented, train_labels_normal))

# Shuffle combined data
combined_indices = np.arange(combined_train_data.shape[0])
np.random.shuffle(combined_indices)

combined_train_data = combined_train_data[combined_indices]
combined_train_labels = combined_train_labels[combined_indices]



val_set = val_datagen.flow_from_directory(val_path,
                                          target_size=(128, 128),
                                          batch_size=batch_size,
                                          class_mode='binary',
                                          shuffle=True)

Found 6426 images belonging to 2 classes.
Found 6426 images belonging to 2 classes.
Found 1252 images belonging to 2 classes.


## Training

In [66]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping


file_path = 'trained_models/vgg16_transfer_learning.keras'

# Define callbacks
checkpoint = ModelCheckpoint(filepath=file_path, verbose=1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

callbacks = [checkpoint, reduce_lr, early_stopping]

# Train the model with callbacks
model_history = model.fit(
    train_set_normal,
    validation_data=val_set,
    epochs=100,
    callbacks=callbacks,
    # verbose=2
)

Epoch 1/100


c:\Users\gusta\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\util\dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


804/804 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.5003
Epoch 1: val_loss improved from inf to 0.00000, saving model to trained_models\vgg16_transfer_learning.keras
804/804 [==============================] - 143s 177ms/step - loss: 0.0000e+00 - accuracy: 0.5003 - val_loss: 0.0000e+00 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 2/100
804/804 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.5000
Epoch 2: val_loss did not improve from 0.00000
804/804 [==============================] - 143s 178ms/step - loss: 0.0000e+00 - accuracy: 0.5000 - val_loss: 0.0000e+00 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 3/100
804/804 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.5000
Epoch 3: val_loss did not improve from 0.00000
804/804 [==============================] - 143s 177ms/step - loss: 0.0000e+00 - accuracy: 0.5000 - val_loss: 0.0000e+00 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 4/100
804/804 [=================

KeyboardInterrupt: 

## Training 1

In [ ]:
# bacth size 16

In [59]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import os

folder_path = 'trained_models'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

checkpoint_path = os.path.join(folder_path, 'vgg16_transfer_learning.keras')

checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=200)  # Stop training if validation loss doesn't improve after 3 epochs

callbacks = [checkpoint, early_stopping]

for layer in model.layers[:-4]:  # Freeze layers up to the last 4 layers
    layer.trainable = False

adam = optimizers.Adam(learning_rate=0.001)  # Smaller learning rate for fine-tuning

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

start = datetime.now()

# Train the model
model_history = model.fit(
    train_set,
    validation_data=val_set,
    epochs=100,  # Adjust the number of epochs as needed
    steps_per_epoch=40,
    validation_steps=8,
    callbacks=callbacks,
    verbose=2
)

# Calculate training duration
duration = datetime.now() - start
print("Training completed in time: ", duration)


Epoch 1/100

Epoch 1: val_loss improved from inf to 0.00000, saving model to trained_models\vgg16_transfer_learning.keras
40/40 - 8s - loss: 0.0000e+00 - accuracy: 0.4437 - val_loss: 0.0000e+00 - val_accuracy: 0.5000 - 8s/epoch - 201ms/step
Epoch 2/100

Epoch 2: val_loss did not improve from 0.00000
40/40 - 7s - loss: 0.0000e+00 - accuracy: 0.4938 - val_loss: 0.0000e+00 - val_accuracy: 0.4375 - 7s/epoch - 183ms/step
Epoch 3/100

Epoch 3: val_loss did not improve from 0.00000
40/40 - 7s - loss: 0.0000e+00 - accuracy: 0.5188 - val_loss: 0.0000e+00 - val_accuracy: 0.5156 - 7s/epoch - 186ms/step
Epoch 4/100

Epoch 4: val_loss did not improve from 0.00000
40/40 - 7s - loss: 0.0000e+00 - accuracy: 0.5094 - val_loss: 0.0000e+00 - val_accuracy: 0.4844 - 7s/epoch - 184ms/step
Epoch 5/100


KeyboardInterrupt: 

## Training 2

In [ ]:
# batch size 8

In [266]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import os

folder_path = 'trained_models'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

checkpoint_path = os.path.join(folder_path, 'vgg16_transfer_learning_2.keras')

checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_best_only=True)

callbacks = [checkpoint, early_stopping]

for layer in model.layers[:-4]:  # Freeze layers up to the last 4 layers
    layer.trainable = False

adam = optimizers.Adam(learning_rate=0.0001)  # Smaller learning rate for fine-tuning

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

start = datetime.now()

# Train the model
model_history = model.fit(
    train_set,
    validation_data=val_set,
    epochs=1000,  # Adjust the number of epochs as needed
    steps_per_epoch=5*6,
    validation_steps=5,
    callbacks=callbacks,
    verbose=2
)

# Calculate training duration
duration = datetime.now() - start
print("Training completed in time: ", duration)


Epoch 1/1000



Epoch 1: val_loss improved from inf to 1.06078, saving model to trained_models\vgg16_transfer_learning_2.keras
30/30 - 6s - loss: 2.3684 - accuracy: 0.7417 - val_loss: 1.0608 - val_accuracy: 0.8250 - 6s/epoch - 205ms/step
Epoch 2/1000

Epoch 2: val_loss did not improve from 1.06078
30/30 - 5s - loss: 2.1783 - accuracy: 0.7917 - val_loss: 2.1636 - val_accuracy: 0.8250 - 5s/epoch - 180ms/step
Epoch 3/1000

Epoch 3: val_loss did not improve from 1.06078
30/30 - 5s - loss: 2.5498 - accuracy: 0.7458 - val_loss: 2.5534 - val_accuracy: 0.8500 - 5s/epoch - 176ms/step
Epoch 4/1000

Epoch 4: val_loss did not improve from 1.06078
30/30 - 6s - loss: 1.7119 - accuracy: 0.8042 - val_loss: 1.8301 - val_accuracy: 0.7750 - 6s/epoch - 186ms/step
Epoch 5/1000

Epoch 5: val_loss did not improve from 1.06078
30/30 - 5s - loss: 1.8761 - accuracy: 0.7778 - val_loss: 1.1746 - val_accuracy: 0.8250 - 5s/epoch - 181ms/step
Epoch 6/1000

Epoch 6: val_loss did not improve from 1.06078
30/30 - 5s - loss: 2.1837 - 

KeyboardInterrupt: 

## Training 3

In [ ]:
# batch size 32
# shuffle True

In [313]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import os

folder_path = 'trained_models'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

checkpoint_path = os.path.join(folder_path, 'vgg16_transfer_learning_3.keras')

checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_best_only=True)

callbacks = [checkpoint]

for layer in model.layers[:-5]:  # Freeze layers up to the last 5 layers
    layer.trainable = False

adam = optimizers.Adam(learning_rate=0.0001)  # Smaller learning rate for fine-tuning

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

start = datetime.now()

# Train the model
model_history = model.fit(
    train_set,
    validation_data=val_set,
    epochs=1000,  # Adjust the number of epochs as needed
    steps_per_epoch=200,
    validation_steps=40,
    callbacks=callbacks,
    verbose=2
)

# Calculate training duration
duration = datetime.now() - start
print("Training completed in time: ", duration)


Epoch 1/1000

Epoch 1: val_loss improved from inf to 2.28163, saving model to trained_models\vgg16_transfer_learning_3.keras
200/200 - 110s - loss: 2.6649 - accuracy: 0.7440 - val_loss: 2.2816 - val_accuracy: 0.7766 - 110s/epoch - 550ms/step
Epoch 2/1000

Epoch 2: val_loss did not improve from 2.28163
200/200 - 111s - loss: 2.3225 - accuracy: 0.7634 - val_loss: 2.5833 - val_accuracy: 0.7312 - 111s/epoch - 555ms/step
Epoch 3/1000

Epoch 3: val_loss improved from 2.28163 to 2.05527, saving model to trained_models\vgg16_transfer_learning_3.keras
200/200 - 111s - loss: 2.2456 - accuracy: 0.7692 - val_loss: 2.0553 - val_accuracy: 0.7594 - 111s/epoch - 554ms/step
Epoch 4/1000

Epoch 4: val_loss did not improve from 2.05527
200/200 - 109s - loss: 2.0175 - accuracy: 0.7823 - val_loss: 2.0791 - val_accuracy: 0.7563 - 109s/epoch - 547ms/step
Epoch 5/1000

Epoch 5: val_loss improved from 2.05527 to 1.75971, saving model to trained_models\vgg16_transfer_learning_3.keras
200/200 - 106s - loss: 1.97

KeyboardInterrupt: 

## Training 4

In [ ]:
# not augmented val data
# batch size 8

In [27]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import os

folder_path = 'trained_models'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

checkpoint_path = os.path.join(folder_path, 'vgg16_transfer_learning_4.keras')

checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_best_only=True)

callbacks = [checkpoint]

for layer in model.layers[:-5]:  # Freeze layers up to the last 5 layers
    layer.trainable = False

adam = optimizers.Adam(learning_rate=0.001)  # Smaller learning rate for fine-tuning

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

start = datetime.now()

# Train the model
model_history = model.fit(
    train_set,
    validation_data=val_set,
    epochs=100,  # Adjust the number of epochs as needed
    steps_per_epoch=12,
    validation_steps=4,
    callbacks=callbacks,
    verbose=2
)

# Calculate training duration
duration = datetime.now() - start
print("Training completed in time: ", duration)


Epoch 1/100

Epoch 1: val_loss improved from inf to 1.31905, saving model to trained_models\vgg16_transfer_learning_4.keras
12/12 - 3s - loss: 1.3100 - accuracy: 0.6146 - val_loss: 1.3191 - val_accuracy: 0.5312 - 3s/epoch - 278ms/step
Epoch 2/100

Epoch 2: val_loss improved from 1.31905 to 0.81115, saving model to trained_models\vgg16_transfer_learning_4.keras
12/12 - 3s - loss: 0.8683 - accuracy: 0.7292 - val_loss: 0.8111 - val_accuracy: 0.8125 - 3s/epoch - 232ms/step
Epoch 3/100

Epoch 3: val_loss improved from 0.81115 to 0.61667, saving model to trained_models\vgg16_transfer_learning_4.keras
12/12 - 3s - loss: 0.6095 - accuracy: 0.7917 - val_loss: 0.6167 - val_accuracy: 0.7188 - 3s/epoch - 227ms/step
Epoch 4/100

Epoch 4: val_loss improved from 0.61667 to 0.33686, saving model to trained_models\vgg16_transfer_learning_4.keras
12/12 - 3s - loss: 0.5122 - accuracy: 0.8021 - val_loss: 0.3369 - val_accuracy: 0.8750 - 3s/epoch - 228ms/step
Epoch 5/100

Epoch 5: val_loss improved from 0.3

KeyboardInterrupt: 

## Validation on validation set

In [318]:
train_path = 'train'
val_path = 'val'
from keras.models import load_model


test_set = val_datagen.flow_from_directory(val_path,
                                            target_size=(128, 128),
                                            batch_size=32,
                                            class_mode='categorical')

print("="*100)
print("Training 1:")
# Load the trained model
model = load_model('trained_models/vgg16_transfer_learning.keras')
# Evaluate the model on the validation set
evaluation_metrics = model.evaluate(test_set)
print("Evaluation Metrics on Validation Set:", evaluation_metrics)


print("="*100)
print("Training 2:")
# Load the trained model
model = load_model('trained_models/vgg16_transfer_learning_2.keras')
# Evaluate the model on the validation set
evaluation_metrics = model.evaluate(test_set)
print("Evaluation Metrics on Validation Set:", evaluation_metrics)



print("="*100)
print("Training 2:")
# Load the trained model
model = load_model('trained_models/vgg16_transfer_learning_3.keras')
# Evaluate the model on the validation set
evaluation_metrics = model.evaluate(test_set)
print("Evaluation Metrics on Validation Set:", evaluation_metrics)

Found 1252 images belonging to 2 classes.


Training 1:
40/40 [==============================] - 19s 466ms/step - loss: 0.6403 - accuracy: 0.8131
Evaluation Metrics on Validation Set: [0.6403016448020935, 0.8130990266799927]
Training 2:
40/40 [==============================] - 19s 455ms/step - loss: 2.0410 - accuracy: 0.7987
Evaluation Metrics on Validation Set: [2.0409765243530273, 0.7987220287322998]
Training 2:
40/40 [==============================] - 19s 459ms/step - loss: 0.3433 - accuracy: 0.8419
Evaluation Metrics on Validation Set: [0.3432927131652832, 0.8418530225753784]


## Validation on blind set

In [11]:
from keras.models import load_model

model = load_model('trained_models/vgg16_transfer_learning_3.keras')

evaluation_data_path = 'test_blind'
evaluation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

evaluation_set = evaluation_datagen.flow_from_directory(evaluation_data_path,
                                                        target_size=(128, 128),
                                                        batch_size=8,
                                                        class_mode=None,
                                                        shuffle=False)


predictions = model.predict(evaluation_set, verbose=1)


print("Predictions:")
print(predictions)




Found 1366 images belonging to 1 classes.
171/171 [==============================] - 25s 143ms/step
Predictions:
[[1.7590761e-06 3.0688559e-05]
 [1.7680595e-09 3.1036787e-10]
 [2.9659876e-01 1.0304617e-01]
 ...
 [5.0589350e-09 1.0259620e-08]
 [9.9671096e-01 9.6899211e-01]
 [1.2098124e-20 2.3636651e-24]]


In [12]:
output_file = open("predictions.txt", "w")
for i, prediction in enumerate(predictions):
    if prediction[0] > 0.5:  # Assuming index 0 corresponds to the "MEL" class
        output_file.write( "NV\n")
    else:
        output_file.write( "MEL\n")
output_file.close()

## Test if it works

In [13]:
with open('predictions.txt') as f:
    guesses = [l.strip() for l in f]
    assert(len(guesses) == 1366)
    assert(sorted(set(guesses)) == ['MEL', 'NV'])

In [14]:
# Open the text file for reading
with open("predictions.txt", "r") as file:
    # Initialize counters
    count_NV = 0
    count_MEL = 0

    # Iterate through each line in the file
    for line in file:
        # Check if the line contains "NV" or "MEL" and increment the corresponding counter
        if "NV" in line:
            count_NV += 1
        elif "MEL" in line:
            count_MEL += 1

# Print the counts
print("Number of NV entries:", count_NV)
print("Number of MEL entries:", count_MEL)


Number of NV entries: 116
Number of MEL entries: 1250


# Finetuned model

In [91]:
import torch.nn as nn

class Finetuned_CNN(nn.Module):
    def __init__(self):
        super(Finetuned_CNN, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=8),
            nn.Conv2d(64, 128, kernel_size=3),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Flatten()
        )


        self.classifier = nn.Sequential(
            nn.Linear(128, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.05),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [317]:
import numpy as np
from torch.utils.data import SubsetRandomSampler, DataLoader

# Shuffle the indices of the dataset
indices = np.arange(len(folder_train))
np.random.shuffle(indices)

# Take the first 100 and last 100 indices
subset_indices = np.concatenate((indices[:100], indices[-100:]))

# Create a DataLoader for the subset
train_loader_subset_2 = DataLoader(combined_dataset, batch_size=8, sampler=SubsetRandomSampler(subset_indices))


In [92]:
def train_model_finetuned(model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs, save_path):
    best_val_acc = 0

    for epoch in range(num_epochs):
        running_loss = 0.0
        model.train()
        for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False):
            optimizer.zero_grad()
            outputs = model(images)

            # Calculate the loss
            loss = criterion(outputs.squeeze(), labels.float())

            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)  # Calculate average loss per sample
        val_acc = evaluate_during_training(model, val_loader)  # Evaluate on validation set

        current_lr = optimizer.param_groups[0]['lr']
        print("Current learning rate:", current_lr)

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc

            torch.save(model.state_dict(), save_path)
            print(f"Model saved at {save_path}")

        scheduler.step(val_acc)

In [97]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

learning_rate = 0.0001  # Initial learning rate
num_epochs = 100  # Number of training epochs
save_path = "trained_models/finetuned_.pth"  # Path to save the trained model
weight_decay = 1e-6  # Weight decay regularization strength

# Initialize model, loss function, optimizer, and scheduler
model_finetuned = Finetuned_CNN()
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model_finetuned.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.05, patience=10)  # Reduce learning rate when a metric has stopped improving

train_model_finetuned(model_finetuned, criterion, optimizer, scheduler, train_loader_augmented, val_loader, num_epochs, save_path)

TypeError: 'module' object is not callable

In [329]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

learning_rate = 0.0001  # Initial learning rate
num_epochs = 1000  # Number of training epochs
save_path = "trained_models/finetuned.pth"  # Path to save the trained model
weight_decay = 1e-6  # Weight decay regularization strength

# Initialize model, loss function, optimizer, and scheduler
model_finetuned = Finetuned_CNN()
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model_finetuned.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=30)  # Reduce learning rate when a metric has stopped improving

train_model_finetuned(model_finetuned, criterion, optimizer, scheduler, train_loader_augmented, val_loader, num_epochs, save_path)

Current learning rate: 0.0001
Epoch 1/1000, Loss: 0.5474, Validation Accuracy: 0.7684
Model saved at trained_models/finetuned.pth


Current learning rate: 0.0001
Epoch 2/1000, Loss: 0.4839, Validation Accuracy: 0.8083
Model saved at trained_models/finetuned.pth


Current learning rate: 0.0001
Epoch 3/1000, Loss: 0.4584, Validation Accuracy: 0.8051


Current learning rate: 0.0001
Epoch 4/1000, Loss: 0.4560, Validation Accuracy: 0.8211
Model saved at trained_models/finetuned.pth


Current learning rate: 0.0001
Epoch 5/1000, Loss: 0.4433, Validation Accuracy: 0.8171


...

Current learning rate: 1.0000000000000004e-08
Epoch 400/1000, Loss: 0.2974, Validation Accuracy: 0.8522


Current learning rate: 1.0000000000000004e-08
Epoch 401/1000, Loss: 0.2941, Validation Accuracy: 0.8466


Current learning rate: 1.0000000000000004e-08
Epoch 402/1000, Loss: 0.2948, Validation Accuracy: 0.8435


Current learning rate: 1.0000000000000004e-08
Epoch 403/1000, Loss: 0.3013, Validation Accuracy: 0.8514


KeyboardInterrupt: 

In [330]:
save_path = "trained_models/finetuned.pth"
model_finetuned = Finetuned_CNN()
model_finetuned.load_state_dict(torch.load(save_path))
model_baseline.eval()

val_acc = evaluate_during_training(model_finetuned, val_loader)
print("Finetuned_CNN validation accuracy: ", val_acc)

Finetuned_CNN validation accuracy:  0.8626198083067093


# Evaluating all models on test set

In [15]:
transform = transforms.Compose([
    transforms.ToTensor()
])



folder_test = ImageFolder('./test', transform=transform)
test_loader = DataLoader(folder_test, batch_size=8, shuffle=True)

## Baseline

In [16]:
save_path = "trained_models/baseline.pth"
model_baseline = Baseline_CNN()
model_baseline.load_state_dict(torch.load(save_path))
model_baseline.eval()

test_acc = evaluate_during_training(model_baseline, test_loader)  # Evaluate on validation set
print("Baseline test accuracy: ", test_acc)

Baseline test accuracy:  0.794289897510981


## Augmented

In [33]:
save_path = "trained_models/augmented_.pth"
model_augmented = Augmented_CNN()
model_augmented.load_state_dict(torch.load(save_path))
model_augmented.eval()

test_acc = evaluate_during_training(model_augmented, test_loader)
print("Augmentated_CNN validation accuracy: ", test_acc)

Augmentated_CNN validation accuracy:  0.8272327964860908


## Batch normalization

In [37]:
save_path = "trained_models/batch_norm.pth"
model_batch_norm = BatchNormalizedCNN()
model_batch_norm.load_state_dict(torch.load(save_path))
model_batch_norm.eval()

test_acc = evaluate_during_training(model_batch_norm, test_loader)
print("BatchNormalized_CNN test accuracy: ", test_acc)

BatchNormalized_CNN test accuracy:  0.8374816983894583


## Residual

In [55]:
save_path = "trained_models/residual.pth"
model_residual = Residual_CNN()
model_residual.load_state_dict(torch.load(save_path))
model_residual.eval()

test_acc = evaluate_during_training(model_residual, test_loader)
print("Residual_CNN test accuracy: ", test_acc)

Residual_CNN test accuracy:  0.7349926793557833


## Transfer learning

In [69]:
import warnings
import tensorflow as tf
warnings.filterwarnings("ignore", category=DeprecationWarning, module="tensorflow")

test_path = 'test'

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size=(128, 128),
                                            batch_size=8,
                                            class_mode='binary',
                                            shuffle=False)

print("="*100)
print("Training 1:")
# Load the trained model
model = load_model('trained_models/vgg16_transfer_learning.keras')
# Evaluate the model on the validation set
evaluation_metrics = model.evaluate(test_set)
print("Evaluation Metrics on Validation Set:", evaluation_metrics)

print("="*100)
print("Training 2:")
# Load the trained model
model = load_model('trained_models/vgg16_transfer_learning_2.keras')
# Evaluate the model on the validation set
evaluation_metrics = model.evaluate(test_set)
print("Evaluation Metrics on Validation Set:", evaluation_metrics)

print("="*100)
print("Training 3:")
# Load the trained model
model = load_model('trained_models/vgg16_transfer_learning_3.keras')
# Evaluate the model on the validation set
evaluation_metrics = model.evaluate(test_set)
print("Evaluation Metrics on Validation Set:", evaluation_metrics)

print("="*100)
print("Training 4:")
# Load the trained model
model = load_model('trained_models/vgg16_transfer_learning_4.keras')
# Evaluate the model on the validation set
evaluation_metrics = model.evaluate(test_set)
print("Evaluation Metrics on Validation Set:", evaluation_metrics)

Found 1366 images belonging to 2 classes.
Training 1:


c:\Users\gusta\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\util\dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


171/171 [==============================] - 26s 150ms/step - loss: 0.0000e+00 - accuracy: 0.5000
Evaluation Metrics on Validation Set: [0.0, 0.5]
Training 2:


ValueError: in user code:

    File "c:\Users\gusta\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 2066, in test_function  *
        return step_function(self, iterator)
    File "c:\Users\gusta\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 2049, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\gusta\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 2037, in run_step  **
        outputs = model.test_step(data)
    File "c:\Users\gusta\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1919, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\gusta\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "c:\Users\g

## Finetuned Model

In [331]:
save_path = "trained_models/finetuned.pth"
model_finetuned = Finetuned_CNN()
model_finetuned.load_state_dict(torch.load(save_path))
model_finetuned.eval()

test_acc = evaluate_during_training(model_finetuned, test_loader)
print("Finetuned_CNN test accuracy: ", test_acc)

Finetuned_CNN test accuracy:  0.8645680819912153
